In [4]:
import os

In [5]:
files = os.listdir('data')

In [6]:
files

['ratings_8.txt',
 'ratings_9.txt',
 'ratings_10.txt',
 'ratings_7.txt',
 'ratings_6.txt',
 'ratings_4.txt',
 'ratings_5.txt',
 'ratings_1.txt',
 'ratings_2.txt',
 'ratings_3.txt']

In [10]:
mixed_files = [

'movies.csv',

'notes.txt',

'ratings.zip',

'ratings_1.txt',

'ratings_10.txt',

'ratings_2.txt',

'ratings_3.txt',

'ratings_6.txt',

'ratings_7.txt',

'ratings_8.txt',

'ratings_9.txt',

'subfolder'

]

In [11]:
answer = [a for a in mixed_files if 'ratings_' in a] #ищем файлы с нужным названием

In [12]:
len(answer)

8

In [13]:
for root, dirs, files in os.walk('data'):
    print('Прохожу файлы папки {}'.format(root))
    print('Файлы в папке: {}'.format(files))
    print('Список внутренних папок: {}\n'.format(dirs))

Прохожу файлы папки data
Файлы в папке: ['ratings_8.txt', 'ratings_9.txt', 'ratings_10.txt', 'ratings_7.txt', 'ratings_6.txt', 'ratings_4.txt', 'ratings_5.txt', 'ratings_1.txt', 'ratings_2.txt', 'ratings_3.txt']
Список внутренних папок: []



In [14]:
level_0 = ['ratings_1.txt', 'ratings_10.txt', 'ratings_2.txt']

level_1 = ['ratings_1_subfolder.txt', 'ratings_2_subfolder.txt', 'ratings_3_subfolder.txt']

level_2 = ['ratings_logs_001.txt', 'ratings_logs_002.txt', 'ratings_logs_003.txt']

In [15]:
a = len(level_0) + len(level_1) + len(level_2)
a

9

Поскольку мы получили только имена файлов, то нам теперь надо указывать методу pd.read_csv не только на имя файла, но и папку 'data', в которой лежит файл. Для того, чтобы объединять названия папок и имена файлов, можно воспользоваться удобным методом os.path.join (в нашем простом случае можно было объединить их через слэш, но в случае длинных адресов этот метод сэкономит вам много времени):

In [16]:
print(os.path.join('data', 'ratings_1.txt'))

data/ratings_1.txt


In [17]:
import pandas as pd
files = os.listdir('data')
data = pd.read_csv( os.path.join('data', files[0]) )
data.head()

,483,1573,2.0,1465387568
0,483,1580,3.0,1465387435
1,483,1676,4.0,1465387586
2,483,1682,3.0,1465387384
3,483,1732,5.0,1465387406
4,483,1884,4.0,1465387578


С помощью библиотеки os можно автоматизировать огромное количество операций с файлами. Например, можно быстро оценить размер папки, из которой мы собираемся брать данные. В случае больших файлов это очень важно, т. к. при чтении файла в датафрейм (без опции построчного чтения) весь объем файла пишется в оперативную память компьютера. С помощью метода getsize можно получить размер файлов в байтах:



In [18]:
from os.path import join, getsize
for root, dirs, files in os.walk('data'):
    print(sum(getsize(join(root, name)) for name in files))

2438233


На прошлом шаге мы получили список файлов в папке и прочитали первый в датафрейм data. Давайте повторим процедуру для всех файлов и склеим все выгрузки в один датафрейм.

В этом нам поможет метод concat.

Этот метод позволяем склеивать два и более датафреймов в один. Например, если у нас есть датафреймы df1 и df2 с одинаковыми столбцами (обратите внимание, что название последнего столбца у них разное):

In [27]:
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'], 

                    'B': ['B0', 'B1', 'B2', 'B3'], 

                    'C': ['C0', 'C1', 'C2', 'C3'], 

                    'D': ['D0', 'D1', 'D2', 'D3']})

In [28]:
df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'], 

                    'B': ['B4', 'B5', 'B6', 'B7'],

                    'C': ['C4', 'C5', 'C6', 'C7'],

                    'E': ['D4', 'D5', 'D6', 'D7']})

In [31]:
frames = [df1, df2]

pd.concat(frames)

,A,B,C,D,E
0,A0,B0,C0,D0,NaN
1,A1,B1,C1,D1,NaN
2,A2,B2,C2,D2,NaN
3,A3,B3,C3,D3,NaN
0,A4,B4,C4,NaN,D4
1,A5,B5,C5,NaN,D5
2,A6,B6,C6,NaN,D6
3,A7,B7,C7,NaN,D7


Метод concat по умолчанию склеивает датафреймы вертикально, самостоятельно определяя одинаковые столбцы. Можно горизонтально

In [32]:
pd.concat([df1, df2], axis=1)

,A,B,C,D,A,B,C,E
0,A0,B0,C0,D0,A4,B4,C4,D4
1,A1,B1,C1,D1,A5,B5,C5,D5
2,A2,B2,C2,D2,A6,B6,C6,D6
3,A3,B3,C3,D3,A7,B7,C7,D7


Итак, ранее мы прочитали содержимое одного файла с рейтингами в датафрейм. Список всех десяти файлов у нас лежит в списке files. Запишем содержимое всех файлов в датафрейм data.

Сначала создадим его:

data = pd.DataFrame(columns = ['userId', 'movieId', 'rating', 'timestamp'])
Для каждого имени файла filename будем записывать его содержимое в датафрейм temp. 

Допустим, имя текущего файла записано в переменную filename, тогда для его импорта в датафрейм используем переменную temp:

temp = pd.read_csv( os.path.join('data', filename), names = ['userId', 'movieId', 'rating', 'timestamp'] )
Наконец, для добавления содержимого очередного файла filename к основному датафрейму data останется использовать метод concat:

data = pd.concat([data, temp])
Использование временного датафрейма temp аналогично тому как мы прибавляем значение к переменной i в цикле: i = i + 1. Только вместо i у нас датафрейм data, а вместо единицы - temp.

Напишите цикл, который собирает содержимое файлов папки data в единый датафрейм data

In [35]:
import os
import pandas as pd
data = pd.DataFrame(columns = ['userId', 'movieId', 'rating', 'timestamp'])
files = os.listdir('data')
for file in files:
    temp = pd.read_csv( os.path.join('data', file), names = ['userId', 'movieId', 'rating', 'timestamp'] )
    data = pd.concat([data, temp])
data.head()

,userId,movieId,rating,timestamp
0,483,1573,2.0,1465387568
1,483,1580,3.0,1465387435
2,483,1676,4.0,1465387586
3,483,1682,3.0,1465387384
4,483,1732,5.0,1465387406


In [36]:
len(data)

100004